# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import DataScienceHelperLibrary as dsh
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
_, files = dsh.ReadCsvFiles(['messages.csv', 'categories.csv'])

-------------------- Start reading files --------------------
Dataframe loaded from messages.csv: shape = (26248, 4)
Dataframe loaded from categories.csv: shape = (26248, 2)
-------------------- Reading files successfully finished --------------------


In [3]:
# load messages dataset
messages = files['messages.csv']
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
dsh.AnalyzeDataFrame(messages)

-------------------- Dataframe analysis started --------------------
Shape:  (26248, 4)
Number of duplicate rows:  68
-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 3, 75.00%
id, type: int64          0.0
message, type: object    0.0
genre, type: object      0.0
dtype: float64
Columns having > 50% and <= 70% missing values: 1, 25.00%
original, type: object    0.612009
dtype: float64
-------------------- Analysis of Columns with NaN values finished --------------------
-------------------- Dataframe analysis finished --------------------


As there are many rows with no original text, let's check what's the difference between those with orig text and those without:

In [5]:
messages[messages['original'].isnull()].head()

,id,message,original,genre
7433,8365,NOTES: It mark as not enough information,NaN,direct
9902,11186,My thoughts and prayers go out to all the live...,NaN,social
9903,11188,I m sorry for the poor people in Haiti tonight...,NaN,social
9904,11189,RT selenagomez UNICEF has just announced an em...,NaN,social
9905,11192,lilithia yes 5.2 magnitude earthquake hit mani...,NaN,social


In [6]:
messages[~messages['original'].isnull()].head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


I assume that the column 'original' keeps the message original text if it is not written in english. If the oiginal text is written in english, it is stored in the message column and the value in the corresponding original cell is NaN (or NULL in database).


Messages starting with 'NOTES:' got my attention, so have a look at them.

In [7]:
ser = messages['message']

ser[ser.str.startswith('NOTES:')]

3069                   NOTES: Personal. Not an emergency.
3102             NOTES: aclet dry We are counting on you.
3106                      NOTES: talking about gas prices
3169                 NOTES: Historic earthquake in Haiti.
3243    NOTES: They are requesting that somebody to pa...
3265    NOTES: Regular gossip or message sharing. Not ...
3275           NOTES: personal request. Not an emergency.
3318             NOTES: regular gossip. Not an emergency.
3364                     NOTES: Gossip. Not an emergency.
3369           NOTES: Personal comment. Not an emergency.
3370           NOTES: Personal opinion. Not an emergency.
3383        NOTES: Inquiry. No action needed at this time
3393               NOTES: looking for work - construction
3396       NOTES: Concerns about remainder of school year
3399           NOTES: Gossip. Personal. Not an emergency.
3400                    NOTES: Discard. Not an emergency.
3402    NOTES: Personal concerns and offer of help to ...
3405          

Most of the messages are like 'Not important', 'Already translated' and so on.
So I decide to drop those rows.

In [8]:
dsh.AnalyzeColumn(messages, 'genre')

-------------------- Analysing column/s "['genre']" --------------------
Datatype (dtype) =  object
-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 1, 100.00%
genre, type: object    0.0
dtype: float64
-------------------- Analysis of Columns with NaN values finished --------------------
Considering columns:  ['genre']
-------------------- Dataframe value counts analye started --------------------
********************  ********************
news      13068
direct    10782
social     2398
Name: genre, dtype: int64
********************  ********************
-------------------- Dataframe value counts analysis finished --------------------
-------------------- Finished analysing column/s "['genre']" --------------------


In [9]:
# load categories dataset
categories = files['categories.csv']
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [10]:
dsh.AnalyzeDataFrame(categories)

-------------------- Dataframe analysis started --------------------
Shape:  (26248, 2)
Number of duplicate rows:  32
-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 2, 100.00%
id, type: int64             0.0
categories, type: object    0.0
dtype: float64
-------------------- Analysis of Columns with NaN values finished --------------------
-------------------- Dataframe analysis finished --------------------


In [11]:
dsh.AnalyzeEqualColumns(messages, categories)

-------------------- Starting comparing dataframes: --------------------
Equal columns found:  ['id']
Column id: All values in both columns contained
Column id: Value counts are equal
-------------------- Finished comparing dataframes: --------------------


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [12]:
# merge datasets
df = dsh.QuickMerge(messages, categories)

dsh.DfTailHead(df, 4)

-------------------- Merging dataframes on columns:  --------------------
['id']
New shape:  (26386, 5)
--------------------  --------------------


,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
26382,30262,A suitable candidate has been selected and OCH...,NaN,news,related-0;request-0;offer-0;aid_related-0;medi...
26383,30263,"Proshika, operating in Cox's Bazar municipalit...",NaN,news,related-1;request-0;offer-0;aid_related-0;medi...
26384,30264,"Some 2,000 women protesting against the conduc...",NaN,news,related-1;request-0;offer-0;aid_related-1;medi...
26385,30265,A radical shift in thinking came about as a re...,NaN,news,related-1;request-0;offer-0;aid_related-0;medi...


In [13]:
print(df.shape)

(26386, 5)


Now there are more rows due to duplicate entries in both files.

In [14]:
dsh.DfTailHead(messages[~messages['original'].isnull()], 4)

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
12366,14673,Ghambat: Sindh: Volunteers needed : @ http://b...,Ghambat: Sindh: Razakaroon ki zaroorat hai : @...,direct
12367,14675,"shikarpur: Volunteers needed, food and medical...",shikarpur mein swanyasevak ki avyashaktha he.k...,direct
12368,14676,"Dadu: Needs water, food, medicines. boats","JO KO KHANA, PAANI, DAWAYIAN, NAOV CHAHIYE.",direct
12370,14679,Umer kot: Sindh: Shelter and Camp needed as pe...,"Umer kot: Sindh: logon ko rehne ke liye ghar,s...",direct


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [15]:
uniqueCategories = dsh.GetUniqueValuesListFromColumn(
    df, 
    'categories', 
    clean = {'' : ['-1', '-0'] },
    splitby = ';',
)

-------------------- Start cleaning values in columns: --------------------
Applied on "categories":  replaced "['-1', '-0']" by ""
-------------------- Finished cleaning values in columns: --------------------
-------------------- Column "categories" has 37 unique values --------------------


In [16]:
_ = dsh.CheckIfValuesContainedInEachOther(uniqueCategories)

-------------------- Following values are contained in others: --------------------
related  -  ['aid_related', 'infrastructure_related', 'weather_related', 'related-2']
--------------------  --------------------


I made this check because I was thinking about an alternative way to encode this column. Just extract the pure category name and check if "{category}-1" in whole category string or not. Depending on that add column with values 1 or 0. But that won't work with 'related' as the output says.

The last value 'related-2' attracted my attention. I assumed that it is in the first column. So I checked it:

In [17]:
df[df['categories'].str.contains('related-2')]

,id,message,original,genre,categories
117,146,Dans la zone de Saint Etienne la route de Jacm...,Nan zon st. etine rout jakmel la bloke se mize...,direct,related-2;request-0;offer-0;aid_related-0;medi...
221,263,. .. i with limited means. Certain patients co...,t avec des moyens limites. Certains patients v...,direct,related-2;request-0;offer-0;aid_related-0;medi...
307,373,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...,direct,related-2;request-0;offer-0;aid_related-0;medi...
462,565,"Bonsoir, on est a bon repos aprs la compagnie ...",Bonswa nou nan bon repo apri teleko nan wout t...,direct,related-2;request-0;offer-0;aid_related-0;medi...
578,700,URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX ...,r et Salon Furterer. mwen se yon Cosmtologue. ...,direct,related-2;request-0;offer-0;aid_related-0;medi...
657,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,related-2;request-0;offer-0;aid_related-0;medi...
658,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,related-2;request-0;offer-0;aid_related-0;medi...
659,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,related-2;request-0;offer-0;aid_related-0;medi...
660,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct,related-2;request-0;offer-0;aid_related-0;medi...
889,1063,no authority has passed by to see us. We don't...,EN OKENN OTORITE POKO PASE WE NOU NOU PAGEN KO...,direct,related-2;request-0;offer-0;aid_related-0;medi...


My first asumption - without having a look at the messages - was that higher numbers are something like a weight (0 does not belong to this category, 1 tendentially/probably yes and 2 definitely. 
But as I saw the messages, my assumption "english text in message column, foreign text - if present, in original column".

There are multiple ways how to treat multiple languages and I decided myself for the bold printed one:
- store in database and use for model,
- #### store in database to have data present and select via sql only those with related = 1 or just drop those rows before training,

- either, or... don't consider and just drop.

In [18]:
uniqueCategories.remove('related-2')

In [19]:
ser = categories['categories']

categories = ser.str.split(';', expand = True)

In [20]:
# select the first row of the categories dataframe
#row = df[:1]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing

category_colnames = uniqueCategories
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [21]:
# rename the columns of `categories`

categories.columns = category_colnames

In [22]:
categories.head(n = 2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [23]:
typelist = []
for column in uniqueCategories:
    # set each value to be the last character of the string
    categories[column] = categories[column].apply(lambda x: int(str(x).replace(column + '-', "")))
    # convert column from string to numeric
    
    # This step does not need to be done beuase I'm already calling int(..) in apply
    typ = str(categories[column].dtype)
    if typ in typelist:
        continue
    typelist.append(typ)
    # data is at this point already numeric
    
print('DType/s of new columns is/are: ', typelist)

DType/s of new columns is/are:  ['int64']


In [24]:
categories.head(n = 2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [25]:
# For Testing
dfcopy = df.copy(deep = True)

In [26]:
# drop the original categories column from `df`
df = df.drop('categories', axis = 1)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [27]:
print(df.shape, categories.shape)

(26386, 4) (26248, 36)


In [28]:
# concatenate the original dataframe with the new `categories` dataframe

#dfFinal = pd.concat([df, categories], axis = 1, sort = None )

dfFinal = df.join(categories, how = 'inner', on = 'id')

dfFinal.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [29]:
# check number of duplicates
# drop duplicates
# check number of duplicates

# These steps are implemented in following function:

dfFinal = dsh.RemoveDuplicateRows(dfFinal)

-------------------- Removing duplicate rows --------------------
Current shape:  (22809, 40)
There are  188  duplicates in the data
All duplicates successfully removed. New size:  (22621, 40)
--------------------  --------------------


As my analysis shows (after loading the file), there are still NaN values in the dataframe.
So before saving values in database, I replace Nan by None (in hope it will be converted to DBNULL).

In [30]:
dfFinal.isnull().sum()

id                            0
message                       0
original                  12468
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [31]:
dfFinal[dfFinal.original.isnull()]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7483,8365,NOTES: It mark as not enough information,NaN,direct,1,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
9956,11186,My thoughts and prayers go out to all the live...,NaN,social,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
9957,11188,I m sorry for the poor people in Haiti tonight...,NaN,social,1,0,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0
9958,11189,RT selenagomez UNICEF has just announced an em...,NaN,social,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9959,11192,lilithia yes 5.2 magnitude earthquake hit mani...,NaN,social,1,0,0,1,0,0,...,0,1,1,1,0,0,0,0,0,0
9960,11193,RT TheNewsBlotter RT caribnews On Call Interna...,NaN,social,1,1,0,1,1,0,...,0,0,1,1,0,0,0,0,0,1
9961,11195,Most Eureka homeowners won&#39 t collect on qu...,NaN,social,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9962,11196,Haiti hit by largest earthquake in over 200 ye...,NaN,social,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9963,11197,RT pinkelephantpun Earthquake Relief Donate ht...,NaN,social,1,0,0,1,0,0,...,0,0,1,0,1,1,0,0,1,1
9964,11198,Praying that today s earthquake in #Haiti mini...,NaN,social,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Remark:

As there is only the column 'original' with nan values, we can just replace them by None so that it is a clear representation in database.

In [32]:
dfDb = dfFinal #.replace(np.nan, None)

In [33]:
dsh.AnalyzeNanColumns(dfDb)

-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 39, 97.50%
id, type: int64                        0.0
message, type: object                  0.0
genre, type: object                    0.0
related, type: int64                   0.0
request, type: int64                   0.0
offer, type: int64                     0.0
aid_related, type: int64               0.0
medical_help, type: int64              0.0
medical_products, type: int64          0.0
search_and_rescue, type: int64         0.0
security, type: int64                  0.0
military, type: int64                  0.0
child_alone, type: int64               0.0
water, type: int64                     0.0
food, type: int64                      0.0
shelter, type: int64                   0.0
clothing, type: int64                  0.0
money, type: int64                     0.0
missing_people, type: int64            0.0
refugees, type: int64                  0.0
death, type: int64      

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

## Remark:  
"Clean" is relative. I just encoded and removed duplicates from matching duplicate ids from categories to duplicate ids from messages (as explored above).

I was deliberating to "clean" data for database by removing rows starting with 'NOTES:' and so on. But I decided against this step because it might be that somehow values appear in database that did not go through that cleaning routine. So I am going to analyze and drop rows that are irreleveant/not processable before using for modeling.

In [34]:
database = 'MyDisasterResponse.db'
table = 'MyMessage'

useOrig = False

#if not useOrig:
#    origDb = 'My' + origDb
#    origMessages = 'My' + origMessages

engine = create_engine('sqlite:///{}'.format(database))

In [35]:
dsh.PrintLine('Current Tables in DB:')
print(engine.table_names())
dsh.PrintLine()

-------------------- Current Tables in DB: --------------------
[]
--------------------  --------------------


In [36]:
sql = 'DROP TABLE IF EXISTS ' + table 
_ = engine.execute(sql)

In [37]:
print('Inserting rows: ', dfFinal.shape[0])

Inserting rows:  22621


sql = 'SELECT * FROM ' + origMessages
tblMessage = engine.execute(sql)

columnNames = tblMessage.keys()

row = tblMessage.fetchall()

print('Current number of rows: ', len(row))
print('Current columns: ', columnNames)

#row[:1]

notexist = []
for dfc in list(df.columns):
    if not dfc in columnNames:
        notexist.append(dfc)
if len(notexist) > 0:
    print('Following columns do not exist in table "Message": ', notexist)
else:
    print('All columns contained in table "Message"')

In [38]:
dfDb.to_sql(table, engine, if_exists = 'append', chunksize = 4)

In [39]:
engine.dispose()

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [40]:
#!tar chvfz ETL_Pipeline_Preparation.tar.gz *